In [23]:
import pandas as pd

In [24]:
user_df = pd.read_csv('/opt/ml/recipe_project/data/sequence_final.csv')
cate_df = pd.read_csv('/opt/ml/recipe_project/data/cate_data_indexing.csv')

user_df['recipeid'] = user_df['rid']
user_df = user_df.drop(['rid','star'], axis=1)

In [38]:
user_df

,uid,time,recipeid
0,1,2017-08-27 18:33,6856432
1,1,2018-08-15 18:17,6885928
2,1,2019-01-08 14:30,6886836
3,1,2019-01-25 19:38,6892249
4,1,2019-02-07 15:05,6849655
...,...,...,...
152764,14982,2019-09-05 11:34,6877808
152765,14983,2019-06-21 22:42,6914132
152766,14983,2019-12-06 19:24,6909646
152767,14983,2021-01-02 18:06,6865345


In [37]:
user_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 152769 entries, 0 to 152768
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   uid       152769 non-null  int64 
 1   time      152769 non-null  object
 2   recipeid  152769 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.5+ MB


In [36]:
cate_df

,recipeid,cat1,cat2,cat3,cat4
0,6912734,13,14,34,55
1,6843136,7,19,36,55
2,7002443,13,15,43,57
3,6996297,13,14,33,44
4,6885909,1,14,33,55
...,...,...,...,...,...
120824,7004885,4,18,37,52
120825,7004887,0,14,29,46
120826,7004898,11,14,29,55
120827,7004906,3,15,42,48


In [39]:
cate_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120829 entries, 0 to 120828
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype
---  ------    --------------   -----
 0   recipeid  120829 non-null  int64
 1   cat1      120829 non-null  int64
 2   cat2      120829 non-null  int64
 3   cat3      120829 non-null  int64
 4   cat4      120829 non-null  int64
dtypes: int64(5)
memory usage: 4.6 MB


In [26]:
print(sorted(cate_df['cat1'].unique()))
cate_df['cat2'] = cate_df['cat2'] + 14
print(sorted(cate_df['cat2'].unique()))
cate_df['cat3'] = cate_df['cat3'] + 28
print(sorted(cate_df['cat3'].unique()))
cate_df['cat4'] = cate_df['cat4'] + 44
print(sorted(cate_df['cat4'].unique()))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]
[14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]
[28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43]
[44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60]


# Train Dataset 전처리

In [27]:
df_encoded = pd.get_dummies(cate_df, columns=["cat1", "cat2", "cat3", "cat4"])
df_encoded = df_encoded.replace({False: 0, True: 1})
# 새로운 열 이름 생성
new_columns = ["recipeid"] + [str(i) for i in range(61)]
# 열 이름 변경
df_encoded.columns = new_columns

user_data_df = pd.merge(user_df, df_encoded, on="recipeid").sort_values(by=["uid", "time"]).reset_index(drop=True)
user_data_df = user_data_df.drop(['time'], axis=1)
user_data_df.columns = ["uid", "recipeid"] + [f"{i}" for i in range(61)]

In [28]:
user_data_df

,uid,recipeid,0,1,2,3,4,5,6,7,...,51,52,53,54,55,56,57,58,59,60
0,1,6856432,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
1,1,6885928,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6886836,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6892249,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,6849655,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152764,14982,6877808,0,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
152765,14983,6914132,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
152766,14983,6909646,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
152767,14983,6865345,0,0,1,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0


In [29]:
df_grouped = user_data_df.groupby("uid").apply(lambda x: x.iloc[:, 2:].cumsum()).reset_index(drop=True)
df_grouped = pd.concat([user_data_df[["uid", "recipeid"]], df_grouped], axis=1)
df_grouped = df_grouped.drop('uid', axis=1)

max_values = df_grouped.iloc[:, 1:].max(axis=1)
df_grouped.iloc[:, 1:] = df_grouped.iloc[:, 1:].div(max_values, axis=0)

df_grouped.iloc[:, 1:] = df_grouped.iloc[:, 1:].round(2)

# 레시피 상호작용이 10회 이하인 유저 제외
df_grouped = df_grouped.groupby('recipeid').filter(lambda x: len(x) > 10)
# 인덱스 초기화하면서 기존 인덱스 삭제
df_grouped = df_grouped.reset_index(drop=True)
# 저장
df_grouped.to_csv("train_data.csv", index=False)

In [30]:
df_grouped

,recipeid,0,1,2,3,4,5,6,7,8,...,51,52,53,54,55,56,57,58,59,60
0,6885928,0.00,0.00,0.00,0.00,0.00,0.00,0.0,1.00,0.0,...,0.0,0.0,0.0,0.0,1.00,0.0,0.00,0.0,0.0,0.0
1,6892249,0.00,0.00,0.00,0.50,0.00,0.00,0.0,0.50,0.0,...,0.0,0.0,0.0,0.0,0.50,0.0,0.00,0.0,0.0,0.0
2,6879215,0.00,0.33,0.00,0.33,0.00,0.33,0.0,0.33,0.0,...,0.0,0.0,0.0,0.0,0.33,0.0,0.00,0.0,0.0,0.0
3,6843983,0.00,0.75,0.00,0.25,0.00,0.25,0.0,0.25,0.0,...,0.0,0.0,0.0,0.0,0.50,0.0,0.25,0.0,0.0,0.0
4,6857197,0.00,0.80,0.00,0.20,0.00,0.20,0.0,0.20,0.0,...,0.0,0.0,0.0,0.0,0.60,0.0,0.20,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92775,6869163,0.17,0.33,0.00,0.00,0.17,0.17,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.67,0.0,0.00,0.0,0.0,0.0
92776,6877808,0.17,0.50,0.00,0.00,0.17,0.17,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.83,0.0,0.00,0.0,0.0,0.0
92777,6909646,0.00,1.00,0.00,0.00,0.00,0.00,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0
92778,6865345,0.00,1.00,0.50,0.00,0.00,0.00,0.0,0.00,0.0,...,0.0,0.0,0.0,0.0,0.50,0.0,0.00,0.0,0.0,0.0


In [14]:
# 레시피 종류
len(df_grouped['recipeid'].unique())

2361

### df_grouped에서 0이 차지하는 비율

In [31]:
zero_counts = (df_grouped == 0).sum().sum()
print(zero_counts / (len(df_grouped) * (len(df_grouped.keys()) - 1)) * 100)

74.4279434162959


# recipe category csv onehot 변경

> mongodb input용 데이터 전처리
>
> recipe category를 미리 onehot데이터로 변경하여 연산 최적화

In [32]:
df = pd.read_csv("train_data.csv")
len(df['recipeid'].unique())

2361

In [33]:
df_encoded = pd.get_dummies(cate_df, columns=["cat1", "cat2", "cat3", "cat4"])
df_encoded = df_encoded.replace({False: 0, True: 1})
# 새로운 열 이름 생성
new_columns = ["recipeid"] + [str(i) for i in range(61)]
# 열 이름 변경
df_encoded.columns = new_columns
# "category_list" 열 생성
df_encoded["category_list"] = df_encoded.apply(lambda row: [row[str(i)] for i in range(61)], axis=1)

# "recipeid", "category_list" 열 선택
df_encoded = df_encoded[["recipeid", "category_list"]]

df_encoded.to_csv("category_onehot.csv", index=False)

In [34]:
df_encoded

,recipeid,category_list
0,6912734,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ..."
1,6843136,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
2,7002443,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
3,6996297,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, ..."
4,6885909,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
...,...,...
120824,7004885,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
120825,7004887,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
120826,7004898,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, ..."
120827,7004906,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
